In [69]:
reg_a = 0
reg_b = 0
reg_c = 0
program = []
lc = 0

with open('input.txt') as f:
    for line in f.readlines():
        if lc == 0:
            reg_a = int(line.split(":")[1].strip())
        elif lc == 1:
            reg_b = int(line.split(":")[1].strip())
        elif lc == 2:
            reg_b = int(line.split(":")[1].strip())
        elif lc == 4:
            program = [int(n) for n in line.split(":")[1].strip().split(",")]
        lc += 1

print(f"reg_a: {reg_a}")
print(f"reg_b: {reg_b}")
print(f"reg_c: {reg_c}")
print(f"program: {program}")

reg_a: 63687530
reg_b: 0
reg_c: 0
program: [2, 4, 1, 3, 7, 5, 0, 3, 1, 5, 4, 1, 5, 5, 3, 0]


In [70]:
class Computer():
    def opcode_to_func(self, opcode):
        return {
            0: self.adv,
            1: self.bxl,
            2: self.bst,
            3: self.jnz,
            4: self.bxc,
            5: self.out,
            6: self.bdv,
            7: self.cdv
        }[opcode]

    def __init__(self, a, b, c, program):
        self.reg_a = a
        self.reg_b = b
        self.reg_c = c
        self.program = program
        self.pc = 0
        self.output = []

    def run(self):
        while self.pc < len(self.program):
            try: 
                self.opcode_to_func(self.program[self.pc])()
            except IndexError:
                print(f"IndexError at pc: {self.pc}")
                break
            # print(self)

        print(self)

    def get_combo_op(self, op):
        return {
            0: 0,
            1: 1,
            2: 2,
            3: 3,
            4: self.reg_a,
            5: self.reg_b,
            6: self.reg_c
        }[op]

    def adv(self):
        # print(f"adv: {self.program[self.pc+1]}")
        operand = self.program[self.pc+1]
        numerator = self.reg_a
        divisor = 2**self.get_combo_op(operand)
        # print(f"numerator: {numerator}, divisor: {divisor}")
        self.reg_a = numerator // divisor
        self.pc += 2

    def bxl(self):
        # print(f"bxl: {self.program[self.pc+1]}")
        operand = self.program[self.pc+1]
        # print(f"{self.reg_b} ^ {operand}")
        self.reg_b = self.reg_b ^ operand
        self.pc += 2

    def bst(self):
        # print(f"bst: {self.program[self.pc+1]}")
        operand = self.program[self.pc+1]
        combo_op = self.get_combo_op(operand)
        # print(f"{combo_op} % 8")
        self.reg_b = combo_op % 8
        self.pc += 2

    def jnz(self):
        # print(f"jnz")
        if self.reg_a == 0:
            # print("reg_a is 0, no jump")
            self.pc += 1
            return
        operand = self.program[self.pc+1]
        self.pc = operand

    def bxc(self):
        self.reg_b = self.reg_b ^ self.reg_c
        self.pc += 2  # even though we don't use the operand, we still need to increment the pc

    def out(self):
        # print(f"out: {self.program[self.pc+1]}")
        operand = self.program[self.pc+1]
        self.output.append(self.get_combo_op(operand) % 8)
        self.pc += 2

    def bdv(self):
        # print(f"bdv: {self.program[self.pc+1]}")
        operand = self.program[self.pc+1]
        numerator = self.reg_a
        divisor = 2**self.get_combo_op(operand)
        # print(f"numerator: {numerator}, divisor: {divisor}")
        self.reg_b = numerator // divisor
        self.pc += 2

    def cdv(self):
        # print(f"adv: {self.program[self.pc+1]}")
        operand = self.program[self.pc+1]
        numerator = self.reg_a
        divisor = 2**self.get_combo_op(operand)
        # print(f"numerator: {numerator}, divisor: {divisor}")
        self.reg_c = numerator // divisor
        self.pc += 2

    def __str__(self):
        return f"""
Computer(a={self.reg_a}, b={self.reg_b}, c={self.reg_c}, program={self.program}, pc={self.pc})
Output: {",".join([str(n) for n in self.output])}
        """

    def __repr__(self):
        return str(self)


c = Computer(reg_a, reg_b, reg_c, program)
print(c)
c.run()
print(c.output)


Computer(a=63687530, b=0, c=0, program=[2, 4, 1, 3, 7, 5, 0, 3, 1, 5, 4, 1, 5, 5, 3, 0], pc=0)
Output: 
        
IndexError at pc: 15

Computer(a=0, b=6, c=3, program=[2, 4, 1, 3, 7, 5, 0, 3, 1, 5, 4, 1, 5, 5, 3, 0], pc=15)
Output: 1,6,7,4,3,0,5,0,6
        
[1, 6, 7, 4, 3, 0, 5, 0, 6]


In [71]:
# If register C contains 9, the program 2,6 would set register B to 1.
c = Computer(0, 0, 9, [2, 6])
c.run()
print(c.reg_b == 1)

# If register A contains 10, the program 5,0,5,1,5,4 would output 0,1,2.
c = Computer(10, 0, 0, [5, 0, 5, 1, 5, 4])
c.run()
print(c.output == [0, 1, 2])

# If register A contains 2024, the program 0,1,5,4,3,0 would output 4,2,5,6,7,7,7,7,3,1,0 and leave 0 in register A.
c = Computer(2024, 0, 0, [0, 1, 5, 4, 3, 0])
c.run()
print(c.output == [4, 2, 5, 6, 7, 7, 7, 7, 3, 1, 0])
print(c.reg_a == 0)
# If register B contains 29, the program 1,7 would set register B to 26.
c = Computer(0, 29, 0, [1, 7])
c.run()
print(c.reg_b == 26)
# If register B contains 2024 and register C contains 43690, the program 4,0 would set register B to 44354.
c = Computer(0, 2024, 43690, [4, 0])
c.run()
print(c.reg_b == 44354)



Computer(a=0, b=1, c=9, program=[2, 6], pc=2)
Output: 
        
True

Computer(a=10, b=0, c=0, program=[5, 0, 5, 1, 5, 4], pc=6)
Output: 0,1,2
        
True
IndexError at pc: 5

Computer(a=0, b=0, c=0, program=[0, 1, 5, 4, 3, 0], pc=5)
Output: 4,2,5,6,7,7,7,7,3,1,0
        
True
True

Computer(a=0, b=26, c=0, program=[1, 7], pc=2)
Output: 
        
True

Computer(a=0, b=44354, c=43690, program=[4, 0], pc=2)
Output: 
        
True
